In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
arr = np.array

In [12]:
stations = pd.read_csv("suggested_locations.csv", dtype={'id': 'category'})
comments = pd.read_csv("suggested_comments.csv", dtype={'id': 'category'})

In [13]:
ncomments = comments.groupby('id').id.count().rename('ncomments') + 1
newdf = stations.join(ncomments, on='id')
newdf.ncomments = locations.ncomments.fillna(1)
comments_by_station = comments.groupby('id').comment.apply(list).rename('comments')
# sns.distplot(ncomments)

In [17]:
from itertools import chain
locations = newdf.join(comments_by_station, on='id').\
                groupby(['lat', 'lng']).\
                agg({
                    'id': 'count',
                    'ncomments': 'sum',
                    'comment': lambda x: ''.join(x.dropna()),
                    'comments': lambda x: list(chain(*x.dropna()))
                }).reset_index().rename(columns={'id': 'nstations'})
locations.comments = locations.apply(lambda r: r.comments + [r.comment], axis=1)
locations = locations.drop('comment', axis=1)

In [ ]:
locations[['lat', 'lng' , 'comments', 'ncomments']].
    rename(columns={'ncomments':'weight'}).
    to_json('latlng.json', orient='records')